<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title"><b>A Sudoku Solver</b></span> by <a xmlns:cc="http://creativecommons.org/ns#" href="http://mate.unipv.it/gualandi" property="cc:attributionName" rel="cc:attributionURL">Stefano Gualandi</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/mathcoding/opt4ds" rel="dct:source">https://github.com/mathcoding/opt4ds</a>.

**NOTE:** Run the following script whenever running this script on a Google Colab.

In [ ]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpk") or os.path.isfile("glpk")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass

# Sudoku
The **Sudoku** is a logic-based combinatorial number-placement puzzle (source: [wikipedia](https://en.wikipedia.org/wiki/Sudoku)). The objective is to fill a $9 \times 9$ grid with digits so that each column, each row, and each of the nine $3 \times 3$ subgrids that compose the grid contain all of the digits from 1 to 9. 

The puzzle setter provides a partially completed grid, which for a well-posed puzzle has a single solution.

Completed games are always an example of a *Latin square* which include an additional constraint on the contents of individual regions.

### Example: Game of the day (22-03-2020)
An example of an instance of the [game of the day](http://www.dailysudoku.com/sudoku/today.shtml) is a s follows:

```
. . . | . 9 4 | 8 . .
. 2 . | . 1 7 | 5 . .
. . 6 | . . . | . 1 .
---------------------
. 6 2 | . . 8 | . . 7
. . . | 3 . 2 | . . .
3 . . | 9 . . | 4 2 .
---------------------
. 9 . | . . . | 6 . .
. . 1 | 7 8 . | . 9 .
. . 3 | 4 5 . | . . .
```

We show next how to solve this puzzle (and any other instance of the game) by using **Integer Linear Programming (ILP)**.

## Integer Linear Programming model
A solution strategy for the Sudoku game can be represented by the following **ILP model**.

**Decision Variables:** The variable $x_{ijk} \in \{0,1\}$ is equal to 1 if in position $(i,j)$ in the grid we set the digit $k$, and it is equal to 0 otherwise. For easy of exposition, we use the set $I,J,K:=\{1,\dots,9\}$.

**Objective function:** Since the problem is a feasibility problem, we can set the objective function equal to a constant value. Otherwise, we can add the sum of every variable, and we will expect an optimal solution of value equal to 81 (this way we avoid also a warning from the solver).

**Constraints:** We introduce the following linear constraints, which encode the puzzle rules:

1. In every position, we can place a single digit:
$$
    \sum_{k \in K} x_{ijk} = 1, \;\; \forall i \in I, \; \forall j \in J
$$
2. Each digit appears once per row:
$$
    \sum_{j \in J} x_{ijk} = 1, \;\; \forall i \in I, \; \forall k \in K
$$
3. Each digit appears once per column:
$$
    \sum_{i \in I} x_{ijk} = 1, \;\; \forall j \in J, \; \forall k \in K
$$
4. Each digit appears once per block $3 \times 3$:
$$
    \sum_{i \in I} \sum_{j \in J} x_{(i_0+i)(j_0+j)k} = 1, \;\; \forall i_0,j_0 \in \{1,4,7\}, \;\forall k \in K
$$
5. The digit in the input data must be fixed to 1:
$$
    x_{ijk} = 1, \;\; \forall i,j \in I \times J \; \mbox{ such that } \; Data[i+1][j+1] = 1
$$

We show next how to implement this model in Pyomo.

## Pyomo implementation
As a first step we import the Pyomo libraries.

In [ ]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory
from pyomo.environ import Binary, RangeSet, ConstraintList

We define the input of the problem as a list of list, where the 0 digit is used to denote an unknown cell value.

In [ ]:
Data= [[0, 0, 0, 0, 9, 4, 8, 0, 0],
        [0, 2, 0, 0, 1, 7, 5, 0, 0],
        [0, 0, 6, 0, 0, 0, 0, 1, 0],
        [0, 6, 2, 0, 0, 8, 0, 0, 7],
        [0, 0, 0, 3, 0, 2, 0, 0, 0],
        [3, 0, 0, 9, 0, 0, 4, 2, 0],
        [0, 9, 0, 0, 0, 0, 6, 0, 0],
        [0, 0, 1, 7, 8, 0, 0, 9, 0],
        [0, 0, 3, 4, 5, 0, 0, 0, 0]]

Then, we create an instance of the  class *ConcreteModel*, and we start to add the *RangeSet* and *Var* corresponding to the index sets and the variables of our model.

In [ ]:
# Create concrete model
model = ConcreteModel()

# Sudoku of size 9x9, with subsquare 3x3
n = 9
model.I = RangeSet(1, n)
model.J = RangeSet(1, n)
model.K = RangeSet(1, n)

# Variables
model.x = Var(model.I, model.J, model.K, within=Binary)

At this point, we set the *dummy* objective function.

In [ ]:
# Objective Function
model.obj = Objective(
    expr = sum(model.x[i,j,k] for i in model.I for j in model.J for k in model.K))

Regarding the constraints, we start with the simpler constraints (1)--(3), which set a single digit per cell, per row and per column.

In [ ]:
# 1. A single digit for each position
model.unique = ConstraintList()
for i in model.I:
    for j in model.J:
        expr = 0
        for k in model.K:
            expr += model.x[i,j,k]
        model.unique.add( expr == 1 )

# 2. Row constraints
model.rows = ConstraintList()
for i in model.I:
    for k in model.K:
        expr = 0
        for j in model.J:
            expr += model.x[i,j,k]
        model.rows.add( expr == 1 )

# 3. Column constraints
model.columns = ConstraintList()
for j in model.J:
    for k in model.K:
        expr = 0
        for i in model.I:
            expr += model.x[i,j,k]
        model.columns.add( expr == 1 )

Finally, we declare the constraint for the 9 submatrices 3x3, with the following constraint.

In [ ]:
# 4. Submatrix constraints
model.blocks = ConstraintList()
S = [1, 4, 7]
for i0 in S:
    for j0 in S:
        for k in model.K:
            expr = 0
            for i in range(3):
                for j in range(3):
                    expr += model.x[i0+i, j0+j,k]
            model.blocks.add( expr == 1 )

In [ ]:
# 5. Fix input data
for i in range(n):
    for j in range(n):
        if Data[i][j] > 0:
            model.x[i+1,j+1,Data[i][j]].fix(1)

At this point, we only need to solve the problem and print the solution in a readable format.

In [ ]:
# Solve the model
sol = SolverFactory('glpk').solve(model, tee=True)

In [ ]:
print(sol)

In [ ]:
# Print objective value of the solution
print("objective value:", model.obj())

In [ ]:
# Print a readable format of the solution
for i in model.I:
    for j in model.J:
        for k in model.K:
            if model.x[i,j,k]() > 0:
                print(k, end="  ")
    print()

### Prettify Solution
As a general recommendation, try to have *pretty* output of your solution that can help **humans** to quickly check visually the solution for likely bugs.

For the Sudoku puzzle, we can use the **matplotlib** as follows.

In [ ]:
def PlotSudoku(x, size=6):
    import matplotlib.pyplot as plt
    import numpy as np

    boardgame = np.zeros((9, 9))

    plt.figure(figsize=(size, size))
    plt.imshow(boardgame, cmap='binary')

    for i, j, k in x:
        if x[i,j,k]() > 0:
            if Data[i-1][j-1] == k:
                plt.text(i-1, j-1, k, fontsize=4*size, color='red',
                     ha='center', va='center')
            else:                
                plt.text(i-1, j-1, k, fontsize=4*size, color='darkblue',
                         ha='center', va='center')
             
    # Prettify output
    for i in range(9):
        plt.axhline(y=i+0.5, color='grey', linestyle='--', alpha=0.5)
        plt.axvline(x=i+0.5, color='grey', linestyle='--', alpha=0.5)
    for i in range(3):
        plt.axhline(y=i*3+2.5, color='grey', linestyle='-', lw=2)
        plt.axvline(x=i*3+2.5, color='grey', linestyle='-', lw=2)

    plt.xticks([])
    plt.yticks([])
    plt.show()

PlotSudoku(model.x)

### Optional Exercise
Can you use this ILP model of Sudoku to develop an instance generator?

> **Enjoy!!**